## NYC Lead Locator

This notebook includes the steps of acquiring and combining the data to be used to predict the locations of lead water service lines in New York City. The NYC Open Data Portal contains a dataset with a list of addresses with labeled water service line material. In addition, other datasets in the NYC Open Data Portal included supplementary information regarding construction year, location, dimensions, and market value of buldings by their Borough/Building/Lot (BBL) code. The property value/tax dataset was quite large and required the use of an API to query relevant parts of the dataset. Then Census Tract data was pulled in from the 2013 American Community Survey (U.S. Census Bureau)

In [2]:
#Import necessary libraries
import pandas as pd
import numpy as np
from shapely.geometry import multipolygon
import shapely.wkt
import geopandas
import sodapy as Socrata
import requests

In [3]:
# Download the data from NYC open data
# This dataset containts a list of addresses in NYC. Labels based primarily on historical records
NYC = pd.read_csv('insight/NYC_WaterConnections.csv')
NYC.head()

,BBL,the_geom,OBJECTID,Address,Material_G,Record_Typ,CityOwned,Shape_Leng,Shape_Area
0,1000010010,MULTIPOLYGON (((-74.01690582604903 40.69335343...,1,1 GOVERNORS ISLAND,Not Lead,OBSERVATION,No,12277.830501,7.550344e+06
1,1000010101,MULTIPOLYGON (((-74.04396208819837 40.69006636...,2,1 LIBERTY ISLAND,Unknown,UNAVAILABLE,No,3940.838665,5.018972e+05
2,1000010201,MULTIPOLYGON (((-74.04001513069849 40.70084115...,3,1 ELLIS ISLAND,Unknown,UNAVAILABLE,No,6306.267382,1.148541e+06
3,1000020001,MULTIPOLYGON (((-74.01202751441055 40.70003725...,4,4 SOUTH STREET,Not Lead,HISTORICAL DATA,Yes,2721.059553,1.008251e+05
4,1000020002,MULTIPOLYGON (((-74.01111163437284 40.70102458...,5,10 SOUTH STREET,Unknown,HISTORICAL DATA,Yes,2411.870163,8.724413e+04


This file has the identifying BBL code for 857,536 NYC addresses. Some of the material is unknown.

In [4]:
total_addresses = NYC.shape[0]
percent_not_lead = NYC.loc[NYC['Material_G']=='Not Lead'].shape[0]/total_addresses
percent_lead = NYC.loc[NYC['Material_G']=='Potential Lead'].shape[0]/total_addresses
percent_unknown = NYC.loc[NYC['Material_G']=='Unknown'].shape[0]/total_addresses
percent_NA = NYC.loc[NYC['Material_G']=='Non-Applicable'].shape[0]/total_addresses

print('The NYC dataset contains', total_addresses, 'addresses with labels: Not Lead, Unknown, Lead, Non-Applicable.')
print('The percentage of addresses labeled Not Lead:', percent_not_lead*100)
print('The percentage of addresses labeled Lead:', percent_lead*100)
print('The percentage of addresses labeled Unknown:', percent_unknown*100)
print('The percentage of addresses labeled Non-applicable:', percent_NA*100)

The NYC dataset contains 857536 addresses with labels: Not Lead, Unknown, Lead, Non-Applicable.
The percentage of addresses labeled Not Lead: 56.081727181132926
The percentage of addresses labeled Lead: 16.133550078364056
The percentage of addresses labeled Unknown: 27.69248171505336
The percentage of addresses labeled Non-applicable: 0.09224102544966042


In [98]:
# Rename "Material_G" as Pipe Material
# Drop unknown labels
NYC.rename(columns={"Material_G": "Pipe Material"}, inplace=True)
NYC_lead=NYC[(NYC['Pipe Material']=='Lead') | (NYC['Pipe Material']=='Not Lead')  ]


To make predictions on the pipe material label, more housing data is required from the NYC Open Data Portal. The following querys property and tax information through the NYC Open Data Portal API.

In [ ]:
# New York City Open Data
# https://data.cityofnewyork.us/City-Government/Property-Valuation-and-Assessment-Data/yjxr-fw8i
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'yjxr-fw8i'
app_token = 'ZK2rqIcpP6zuUNuyUQA0Qe6Mr'

client = Socrata.Socrata(socrata_domain, app_token)



In [ ]:
# Here we match the list of labeled addresses with Property and Tax Information updated in January 2020. 
# This includes building value, dimensions, location, census tract, tax class (building type).

def nyc_api

df_2019 = pd.DataFrame()
# Use maximum query limit (50000)
loop_size = 50000
num_loops = 151

for i in range(num_loops):
    results = client.get(socrata_dataset_identifier,
                         select = 'avland, avland2, avtot, avtot2, bble, bin, blddepth, bldfront, census_tract, community_board, council_district, easement, exland,exland2, extot, extot2, fullval, ltdepth, ltfront, nta, owner, stories, taxclass, valtype, zip',
                         limit=loop_size,
                         offset=loop_size * i)
    df_query = pd.DataFrame.from_dict(results)
    df_query.drop_duplicates(subset='bble', keep='first', inplace=True)
    df_2019 = df_2019.append(df_query[df_query['bble'].isin(list(bbl.astype(str)))],sort=False, ignore_index=False)

There was also a NYC building data set through the NYC Open Data Portal which contains construction year. We will now merge all of the Open Portal data.

In [6]:
# Every building in NYC is identified by BBL code (borough, building, lot)

# The building data set includes construction year for each building by BBL
building = pd.read_csv("insight/NYC_building.csv")

# Data from API
df_2019 = pd.read_csv("insight/df_2019_unique.csv")

temp_lead = NYC_lead
temp_building = building
temp_property = df_2019
print('Shape lead:',temp_lead.shape )
print('Shape building:', temp_building.shape)
print('Shape property:', temp_property.shape)

Shape lead: (480921, 9)
Shape building: (1084844, 15)
Shape property: (619185, 26)


In [112]:
# Rename all BBL
temp_building = temp_building.rename(columns={"BASE_BBL": "BBL"})
temp_property = temp_property.rename(columns={"bble":"BBL"})

In [116]:
# Merge by BBL on labeled lead dataset.
m = pd.merge(temp_lead,temp_property,on='BBL',how='left')
n = pd.merge(m,temp_building,on='BBL',how='left')

merged_data = n.drop_duplicates(subset='BBL',keep='first')
#Building data may have multiple entries for units in buildings but each have same BBL and construction year.
merged_data.shape

(480921, 48)

I also brought in demographic data from the 2013 U.S. Census Bureau American Community Survey at the census tract level (approx 15000 people). Pulled Income, poverty, and racial demographics. The datasets were grouped by county, then within each county I grouped by census tract. Which can be downloaded at the ACS website.

In [11]:
Manhattan_income = pd.read_csv('insight/Manhattan_ACS_median_income.csv')
Manhattan_poverty = pd.read_csv('insight/Manhattan_ACS_poverty.csv')
Manhattan_race = pd.read_csv('insight/Manhattan_ACS_race.csv')
Manhattan_hispanic = pd.read_csv('insight/Manhattan_hispanic.csv')

Bronx_income = pd.read_csv('insight/Bronx_ACS_median_income.csv')
Bronx_poverty = pd.read_csv('insight/Bronx_ACS_poverty.csv')
Bronx_race = pd.read_csv('insight/Bronx_ACS_race.csv')
Bronx_hispanic = pd.read_csv('insight/Bronx_hispanic.csv')

Brooklyn_income = pd.read_csv('insight/Brooklyn_ACS_median_income.csv')
Brooklyn_poverty = pd.read_csv('insight/Brooklyn_ACS_poverty.csv')
Brooklyn_race = pd.read_csv('insight/Brooklyn_ACS_race.csv')
Brooklyn_hispanic = pd.read_csv('insight/Brooklyn_hispanic.csv')

Queens_income = pd.read_csv('insight/Queens_ACS_median_income.csv')
Queens_poverty = pd.read_csv('insight/Queens_ACS_poverty.csv')
Queens_race = pd.read_csv('insight/Queens_ACS_race.csv')
Queens_hispanic = pd.read_csv('insight/Queens_hispanic.csv')

StatenIsland_income = pd.read_csv('insight/StatenIsland_ACS_median_income.csv')
StatenIsland_poverty = pd.read_csv('insight/StatenIsland_ACS_poverty.csv')
StatenIsland_race = pd.read_csv('insight/StatenIsland_ACS_race.csv')
StatenIsland_hispanic = pd.read_csv('insight/StatenIsland_hispanic.csv')

In [7]:
Manhattan_income.head(2)

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,...,HC02_EST_VC39,HC02_MOE_VC39,HC01_EST_VC40,HC01_MOE_VC40,HC02_EST_VC40,HC02_MOE_VC40,HC01_EST_VC41,HC01_MOE_VC41,HC02_EST_VC41,HC02_MOE_VC41
0,Id,Id2,Geography,Total; Estimate; Households,Total; Margin of Error; Households,Median income (dollars); Estimate; Households,Median income (dollars); Margin of Error; Hous...,Total; Estimate; Households - One race-- - White,Total; Margin of Error; Households - One race-...,Median income (dollars); Estimate; Households ...,...,Median income (dollars); Estimate; PERCENT IMP...,Median income (dollars); Margin of Error; PERC...,Total; Estimate; PERCENT IMPUTED - Family inco...,Total; Margin of Error; PERCENT IMPUTED - Fami...,Median income (dollars); Estimate; PERCENT IMP...,Median income (dollars); Margin of Error; PERC...,Total; Estimate; PERCENT IMPUTED - Nonfamily i...,Total; Margin of Error; PERCENT IMPUTED - Nonf...,Median income (dollars); Estimate; PERCENT IMP...,Median income (dollars); Margin of Error; PERC...
1,1400000US36061000100,36061000100,"Census Tract 1, New York County, New York",0,12,-,**,-,**,-,...,(X),(X),-,(X),(X),(X),-,(X),(X),(X)


Looking at the ACS data, I will need to rename the columns appropriately.

In [12]:
#Pull relevant columns and rename according to descriptions in dataset

# rename functions
def rename_income(income):
    income = income[['GEO.display-label','HC02_EST_VC02']]
    income.rename(columns={"HC02_EST_VC02": "Median_Income", 'GEO.display-label':'census_tract'},inplace=True)
    return(income)

def rename_poverty(poverty):
    poverty = poverty[['GEO.display-label', 'HC03_EST_VC01']]
    poverty.rename(columns={"HC03_EST_VC01": "percent_below_poverty", 'GEO.display-label':'census_tract'},inplace=True)
    return(poverty)

def rename_race(race):
    race = race[['GEO.display-label','HD01_VD01','HD01_VD02','HD01_VD03','HD01_VD05','HD01_VD08']]
    race.rename(columns={'HD01_VD01':'Total_pop','HD01_VD02':'white','HD01_VD03':'black','HD01_VD05':'asian',
                               'HD01_VD08':"mixed", 'GEO.display-label':'census_tract'},inplace=True)
    return(race)

def rename_hispanic(hispanic):
    hispanic = hispanic[['GEO.display-label','HD01_VD01','HD01_VD02','HD01_VD03']]
    hispanic.rename(columns={'HD01_VD01':'total_pop_hisp_data','HD01_VD02':'hispanic','HD01_VD03':'non_hispanic', 
                                   'GEO.display-label':'census_tract'},inplace=True)
    return(hispanic)

Manhattan_income = rename_income(Manhattan_income)
Bronx_income = rename_income(Bronx_income)
Brooklyn_income = rename_income(Brooklyn_income)
Queens_income = rename_income(Queens_income)
StatenIsland_income = rename_income(StatenIsland_income)

Manhattan_poverty = rename_poverty(Manhattan_poverty)
Bronx_poverty = rename_poverty(Bronx_poverty)
Brooklyn_poverty = rename_poverty(Brooklyn_poverty)
Queens_poverty = rename_poverty(Queens_poverty)
StatenIsland_poverty = rename_poverty(StatenIsland_poverty)

Manhattan_race = rename_race(Manhattan_race)
Bronx_race = rename_race(Bronx_race)
Brooklyn_race = rename_race(Brooklyn_race)
Queens_race = rename_race(Queens_race)
StatenIsland_race = rename_race(StatenIsland_race)

Manhattan_hispanic = rename_hispanic(Manhattan_hispanic)
Bronx_hispanic = rename_hispanic(Bronx_hispanic)
Brooklyn_hispanic = rename_hispanic(Brooklyn_hispanic)
Queens_hispanic = rename_hispanic(Queens_hispanic)
StatenIsland_hispanic = rename_hispanic(StatenIsland_hispanic)



C:\Users\Eric\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [14]:
# Merge on Census Tract

def merge_on_tract(poverty, income, race, hispanic):
    x = pd.merge(poverty, income
        ,how='outer',on='census_tract')
    x = pd.merge(x, race
        ,how='outer',on='census_tract')
    x = pd.merge(x, hispanic
        ,how='outer',on='census_tract')
    return(x)

x_man = merge_on_tract(Manhattan_poverty, Manhattan_income, Manhattan_race, Manhattan_hispanic)
x_brx = merge_on_tract(Bronx_poverty, Bronx_income, Bronx_race, Bronx_hispanic)
x_bkn = merge_on_tract(Brooklyn_poverty, Brooklyn_income, Brooklyn_race, Brooklyn_hispanic)
x_qns = merge_on_tract(Queens_poverty, Queens_income, Queens_race, Queens_hispanic)
x_st = merge_on_tract(StatenIsland_poverty, StatenIsland_income, StatenIsland_race, StatenIsland_hispanic)


In [75]:
ACS_data = pd.DataFrame()
ACS_data = x_man.append(x_brx,ignore_index=True, sort=False)
ACS_data = ACS_data.append(x_bkn,ignore_index=True, sort=False)
ACS_data = ACS_data.append(x_qns,ignore_index=True, sort=False)
ACS_data = ACS_data.append(x_st,ignore_index=True, sort=False)
ACS_data = ACS_data.drop(columns = ['GEO.id','GEO.id2','HD02_VD01','HD02_VD02','HD02_VD03'])
ACS_data = ACS_data.drop(ACS_data.index[0]) # Row for describing column names

In [76]:
# Census Tract mus be reformatted before combining with NYC dats
ACS_data.census_tract.head()

1       Census Tract 1, New York County, New York
2    Census Tract 2.01, New York County, New York
3    Census Tract 2.02, New York County, New York
4       Census Tract 5, New York County, New York
5       Census Tract 6, New York County, New York
Name: census_tract, dtype: object

In [94]:
ACS_data['census_tract'] = pd.to_numeric(ACS_data['census_tract'],errors='coerce')
ACS_data['census_tract'] = np.floor(ACS_data['census_tract']).astype(int)
ACS_data.census_tract.head()

1    1
2    2
3    2
4    5
5    6
Name: census_tract, dtype: int32

## Feature Engineering

Here I include geospatial feature engineering. First the lead index which computes the proportion of lead labels within a 1/4 km radius around each building. Then, a nearest neighbor comparison which computes, for each buildin, the relative distance to the closest nonlead and lead service lines to compare which is the nearest neighbor.

In [122]:
from shapely import wkt
import geopandas as gp
from shapely.geometry import Polygon, Point

locations = merged_data['the_geom_x']
merged = merged_data.copy()
merged['the_geom'] = locations.apply(wkt.loads)
merged['longitude'] = gp.GeoSeries(merged['the_geom']).centroid.x
merged['latitude'] = gp.GeoSeries(merged['the_geom']).centroid.y

gdf = geopandas.GeoDataFrame(
    merged, geometry=[Point(x, y) for x, y in zip(merged.longitude, merged.latitude)])


In [ ]:
# Find proportion of lead in 0.25 km radius for each building

def lead_index_quarter(p):
    # Lat/lon radius of 0.25 km in NYC
    delta_lat_quarter = .0225 
    delta_lon_quarter = .002965
    px = p.x
    py=p.y
    cts = gdf[(gdf['latitude']<py+delta_lat_quarter) & (gdf['longitude']<px+delta_lon_quarter) & (gdf['latitude']>py-delta_lat_quarter) & (gdf['longitude']>px-delta_lon_quarter)]['Pipe Material'].value_counts(normalize=True)
    if (cts.shape[0]==2): return(cts[1])
    elif (cts.index[0]=='Lead'): return(1)
    else: return(0)

gdf = gdf.geometry.apply(lead_index_quarter)


Next we compute nearest neighbor comparison. 

In [ ]:
# The distance between two coordinates varies by location on Earth. Need to convert to km.

def haversine(coord1, coord2):
    import math
    
    # Coordinates in decimal degrees (e.g. 40.2, -73.9)
    lon1, lat1 = coord1.x, coord1.y
    lon2, lat2 = coord2.x, coord2.y
    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    return(km)


In [ ]:
# Compute Nearest neighbor comparison. Dist(Closest Nonlead) - Dist(closest lead)

%%time
# Check in small radius around each building, expand if necessary. 
# This is to avoid comparing buildings we know to be far apart

delta_lat = .0009
delta_lon = .00012


temp = geo_frame


    for index, row in geo_frame.iterrows():
    
        point = row.geometry
        px = point.x
        py = point.y
        temp = geo_frame[(geo_frame['latitude']<py+delta_lat) & (geo_frame['longitude']<px+delta_lon) & (geo_frame['latitude']>py-delta_lat) & (geo_frame['longitude']>px-delta_lon)]
    
        while (len(temp.PipeMaterial.drop(index, axis=0).value_counts()) != 2):
            delta_lat= 2*delta_lat
            delta_lon= 2*delta_lon
            temp = geo_frame[(geo_frame['latitude']<py+delta_lat) & (geo_frame['longitude']<px+delta_lon) & (geo_frame['latitude']>py-delta_lat) & (geo_frame['longitude']>px-delta_lon)]
    


        if (row.PipeMaterial == 'Lead'):
            multipoint = temp[temp['PipeMaterial'] == 'Lead'].drop(index, axis=0).geometry.unary_union
            queried_geom, nearest_geom = nearest_points(point, multipoint)
            geo_frame.loc[index, 'nearest_LSL_dist'] = haversine(point,nearest_geom)
    
            multipoint = temp[temp['PipeMaterial'] == 'Not Lead'].geometry.unary_union
            queried_geom, nearest_geom = nearest_points(point, multipoint)
            geo_frame.loc[index, 'nearest_NLSL_dist'] = haversine(point,nearest_geom)
        elif (row.PipeMaterial == 'Not Lead'):
            multipoint = temp[temp['PipeMaterial'] == 'Lead'].geometry.unary_union
            queried_geom, nearest_geom = nearest_points(point, multipoint)
            geo_frame.loc[index, 'nearest_LSL_dist'] = haversine(point,nearest_geom)
    
            multipoint = temp[temp['PipeMaterial'] == 'Not Lead'].drop(index, axis=0).geometry.unary_union
            queried_geom, nearest_geom = nearest_points(point, multipoint)
            geo_frame.loc[index, 'nearest_NLSL_dist'] = haversine(point,nearest_geom)

    
